# An automatic diagnosis software for detecting the degree of diabetic retinopathy
by using a subset of the APTOS 2019 dataset

In [1]:
from PIL import Image
import IPython.display
import torchvision.transforms as transforms
import torchvision.models as models
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

device = 'cuda' if torch.cuda.is_available() else 'cpu'

## Load Model

In [2]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

predict_model = models.resnet18()
predict_model.fc = nn.Linear(512, 5)

predict_model.to(device)
predict_model.load_state_dict(torch.load('resnet18_pretrained'))
print('Model: ResNet18 with pre-trained intial weights')

Model: ResNet18 with pre-trained intial weights


## Predict result

There are 5 class and each image is rated for the severity of diabetic retinopathy on a scale of 0 to 4.

class 0: No DR

class 1: Mild

class 2: Moderate

class 3: Severe

class 4: Proliferative DR

In [ ]:
# input the image path
# you can choose (just for show):
# show_images/show0.png      (class 0)
# show_images/show1.png      (class 1)
# show_images/show2.png      (class 2)
# show_images/show3.png      (class 3)
# show_images/show4.png      (class 4)
path_img = input()
print("Input image:")
IPython.display.display(IPython.display.Image(path_img, width=250, height=200))
img = Image.open(path_img).convert('RGB')
img = transform(img).view(1, 3, 224, 224)
img = img.to(device)

# predict
print('There are 5 class and each image is rated for the severity of diabetic retinopathy on a scale of 0 to 4.')
predict_model.eval()
with torch.no_grad():
    output = predict_model(img)
    probabilities = F.softmax(output, dim=1).view(-1).cpu().numpy().tolist()
    print('The probabilities of each classes:')
    p_dict = {'0 No DR': probabilities[0], '1 Mild': probabilities[1],
    '2 Moderate': probabilities[2], '3 Severe': probabilities[3], '4 Proliferative DR': probabilities[4]}
    print(p_dict)
    predicted = probabilities.index(max(probabilities))
    print("The most likely result:")
    print('class:', predicted)
    probabilities[predicted] = 0
    predicted = probabilities.index(max(probabilities))
    print("The second possible result:")
    print('class:', predicted)